# Capstone Project - Which Shoe is the Best for You?

General Assembly passion project. Scrape or obtain data from resources online to develop a dataset to perform cleaning, EDA, and analysis on. Try to predict a model on common themes like: 

- Price
- If item is in category A or B
- Cluster and create groups
- Recommender

In [126]:
# Import libraries
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import requests
from time import sleep, strftime

**Testing one website and finding appropriate keys**

Then run again on page 2.

In [121]:
result = requests.get('https://stockx.com/api/browse?page=1&category=152')
json_res = result.json()

print json_res['Products'][0]['shortDescription']
print json_res['Products'][0]['retailPrice']

Adidas-Yeezy-Powerphase-Calabasas-Core-White
120


In [122]:
result = requests.get('https://stockx.com/api/browse?page=2&category=152')
json_res = result.json()

print json_res['Products'][0]['shortDescription']
print json_res['Products'][0]['retailPrice']

Adidas-EQT-Support-Future-Overkill-Coat-Of-Arms
150


### Functions to run for loop to scrape website

First function scrapes the web for an amount of pages (default 50). It will save the raw data as a csv as well. The second function will then clean the dataframe by taking out the unused columns.

In [148]:
def shoe_scraper(pages=50):
    '''Returns one dataframe of all results. 
    And will save into a new file.'''
    
    # Have to run requests first to get appropriate column names
    req = requests.get('https://stockx.com/api/browse?page=1&category=152')
    json_req = req.json()
    df = pd.DataFrame([], columns=json_req['Products'][0].keys())

    for i in range(1,pages):
        try:
            html = 'https://stockx.com/api/browse?page=' + str(int(i)) + '&category=152'
            result = requests.get(html)
            json_res = result.json()
            df = pd.concat([df, pd.DataFrame(json_res['Products'])])
            sleep(0.5)
        except:
            break
    
    # Drop row duplicates
    df.drop_duplicates(['shortDescription', 'urlKey'], inplace=True) 
    
    # Function to save as csv file under today's day as raw (before dropping)
    def csv_maker(df):
        filename = 'StockX_' + strftime("%m%d")
        df.to_csv(filename, encoding='utf-8')
    
    csv_maker(df)
    
    return df

In [149]:
def clean_df(busy_dataframe):
    '''Dropping columns that hold little to no information.
    Then reset the index since we are getting repeated indices'''
    
    busy_dataframe.drop(['breadcrumbs', 'childId', 'countryOfManufacture', 'type', 'uuid', 'dataType', 
             'doppelgangers', 'condition', 'description', 'hidden', 'ipoDate', 'productCategory', 
             'shoeSize', 'urlKey', 'charityCondition', 'releaseTime', 'shortDescription',
             'media', '_highlightResult', 'market'], axis=1, inplace=True)

    # Reset the index, since we are getting repeated indices
    busy_dataframe.reset_index(drop=True, inplace=True)
    return busy_dataframe

In [146]:
scraped_shoe = shoe_scraper()
shoe_df = clean_df(scraped_shoe)